In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os, sys

my_path = '/content/package'
save_path = '/content/drive/MyDrive/Colab Notebooks/package'

# colab notebooks 안에 package 폴더에 패키지 저장

os.symlink(save_path, my_path)

sys.path.insert(0, my_path)

In [1]:
import pandas as pd
import numpy as np
import warnings
import re
from tqdm import tqdm
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
# from scipy.stats import chi2_contingency

warnings.filterwarnings('ignore')

dir_path = '/content/drive/MyDrive/분석 연습/kaggle/'

# 유형별 매출 전처리

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/분석 연습/kaggle/화장품_부문_유형별_매출액_2020_2021.csv', encoding='utf8')

df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   화장품부문유형별(1)  16 non-null     object
 1   2020         16 non-null     object
 2   2020.1       16 non-null     object
 3   2020.2       16 non-null     object
 4   2020.3       16 non-null     object
 5   2020.4       16 non-null     object
 6   2021         16 non-null     object
 7   2021.1       16 non-null     object
 8   2021.2       16 non-null     object
 9   2021.3       16 non-null     object
 10  2021.4       16 non-null     object
dtypes: object(11)
memory usage: 1.5+ KB


,화장품부문유형별(1),2020,2020.1,2020.2,2020.3,2020.4,2021,2021.1,2021.2,2021.3,2021.4
0,화장품부문유형별(1),사례수 (개),매출액(합계) (백만원),매출액(합계) (백만원),국내매출액 (백만원),수출액 (백만원),사례수 (개),매출액(합계) (백만원),매출액(합계) (백만원),국내매출액 (백만원),수출액 (백만원)
1,화장품부문유형별(1),소계,소계,비중 (%),소계,소계,소계,소계,비중 (%),소계,소계
2,전체,8404,40119810,100.0,32127465,7992345,10235,42090051,100.0,34316643,7773408
3,기초화장품 제품류,6391,27694932,69.0,22125202,5569730,8064,31551252,75.0,26452644,5098608
4,색조 화장용 제품류,826,3412784,8.5,2629108,783676,323,3534236,8.4,2359037,1175199
5,두발용 제품류,1489,1913805,4.8,1554669,359136,617,1019546,2.4,897649,121897
6,인체세정용 제품류,1351,2700554,6.7,2364847,335707,1108,3107811,7.4,2441362,666449
7,눈 화장용 제품류,309,333763,0.8,196609,137154,79,983218,2.3,524023,459194
8,두발 염색용 제품류,256,475578,1.2,354110,121468,152,163768,0.4,147897,15871
9,면도용 제품류,97,191758,0.5,173281,18477,17,71016,0.2,61426,9590


In [ ]:
# 컬럼명 변경

df.columns = df.iloc[0,:]

df.columns

Index(['화장품부문유형별(1)', '사례수 (개)', '매출액(합계) (백만원)', '매출액(합계) (백만원)',
       '국내매출액 (백만원)', '수출액 (백만원)', '사례수 (개)', '매출액(합계) (백만원)', '매출액(합계) (백만원)',
       '국내매출액 (백만원)', '수출액 (백만원)'],
      dtype='object', name=0)

In [ ]:
drop_list = [df.columns[i] for i in [2,3,7,8]]

df = df.drop(columns=drop_list, axis=1) # 비중과 합계 컬럼 제거

In [ ]:
# 첫 번째, 두 번째 행 drop
df.drop(index=list(range(0,3)), axis=0, inplace=True)

In [ ]:
df.reset_index(drop=True, inplace=True) # 인덱스 재설정

In [ ]:
df.rename({"화장품부문유형별(1)":"화장품 유형"}, axis=1, inplace=True) # 컬럼명 변경

df.columns

Index(['화장품 유형', '사례수 (개)', '국내매출액 (백만원)', '수출액 (백만원)', '사례수 (개)',
       '국내매출액 (백만원)', '수출액 (백만원)'],
      dtype='object', name=0)

In [ ]:
# 데이터 연도별로 분리 후 concat

df_2020 = df.iloc[:,1:4]
df_2021 = df.iloc[:,4:]

df_2020['연도'] = 2020
df_2021['연도'] = 2021

df_2020.insert(0,df.columns[0],df["화장품 유형"])
df_2021.insert(0,df.columns[0],df["화장품 유형"])

df_new = pd.concat([df_2020, df_2021], axis=0)

In [ ]:
# 데이터 타입 변경

dict_types = {df_new.columns[i] : 'int64' for i in range(1,5)}

df_new = df_new.astype(dict_types)

df_new.dtypes

0
화장품 유형         object
사례수 (개)         int64
국내매출액 (백만원)     int64
수출액 (백만원)       int64
연도              int64
dtype: object

In [ ]:
df_new.to_csv('/content/drive/MyDrive/분석 연습/kaggle/연도별_화장품 유형별 매출.csv', index=False, header=True)

# 유통 채널별 매출

In [ ]:
df_ch = pd.read_csv("/content/drive/MyDrive/분석 연습/kaggle/화장품_부문_유통채널별_매출_2020_2021.csv")

df_ch.info()

df_ch.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   시점             34 non-null     int64 
 1   화장품부문유통채널별(1)  34 non-null     object
 2   화장품부문유통채널별(2)  34 non-null     object
 3   매출액 (백만원)      34 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 1.2+ KB


,시점,화장품부문유통채널별(1),화장품부문유통채널별(2),매출액 (백만원)
0,2020,오프라인 판매,소계,15497034
1,2020,오프라인 판매,백화점,510677
2,2020,오프라인 판매,브랜드숍,493572
3,2020,오프라인 판매,편집숍,3141563
4,2020,오프라인 판매,대형마트/창고형 할인 매장/슈퍼마켓/편의점,847184


In [ ]:
# 컬럼명 변경

df_ch.columns = ["연도", "채널 (중)", "채널 (소)", "매출액 (백만원)"]

df_ch.columns

Index(['연도', '채널 (중)', '채널 (소)', '매출액 (백만원)'], dtype='object')

In [ ]:
# 소계 제거

df_ch.drop(index = df_ch[df_ch['채널 (소)'] == '소계'].index, axis=0, inplace=True)

df_ch[df_ch['채널 (소)'] == '소계']

,연도,채널 (중),채널 (소),매출액 (백만원)


In [ ]:
# 데이터 타입 변경

dict_dtypes2 = {df_ch.columns[i] : 'int64' for i in [0,3]}

df_ch = df_ch.astype(dict_dtypes2)

df_ch.dtypes

연도            int64
채널 (중)       object
채널 (소)       object
매출액 (백만원)     int64
dtype: object

In [ ]:
df_ch_2020 = df_ch[df_ch["연도"]==2020]

df_ch2 = pd.merge(left=df_ch, right=df_ch_2020, on=["채널 (중)", "채널 (소)"], suffixes=['_현재', "_전년"])

print(df_ch.shape[0])
print(df_ch2.shape[0])

# 이전 년도와의 비율 차이 계산

df_ch2 = df_ch2[['연도_현재', '채널 (중)', '채널 (소)', '매출액 (백만원)_현재', '매출액 (백만원)_전년']]

df_ch2["전년도 비율 차이"] = (df_ch2['매출액 (백만원)_현재'] - df_ch2['매출액 (백만원)_전년']) / df_ch2['매출액 (백만원)_전년']

df_ch2 = df_ch2[['연도_현재', '채널 (중)', '채널 (소)', '매출액 (백만원)_현재', "전년도 비율 차이"]]

df_ch2.columns = ["연도", '채널 (중)', '채널 (소)', '매출액 (백만원)', "전년도 비율 차이"]

df_ch2

30
30


,연도,채널 (중),채널 (소),매출액 (백만원),전년도 비율 차이
0,2020,오프라인 판매,백화점,510677,0.000000
1,2021,오프라인 판매,백화점,544048,0.065347
2,2020,오프라인 판매,브랜드숍,493572,0.000000
3,2021,오프라인 판매,브랜드숍,884289,0.791611
4,2020,오프라인 판매,편집숍,3141563,0.000000
5,2021,오프라인 판매,편집숍,2346426,-0.253102
6,2020,오프라인 판매,대형마트/창고형 할인 매장/슈퍼마켓/편의점,847184,0.000000
7,2021,오프라인 판매,대형마트/창고형 할인 매장/슈퍼마켓/편의점,1429238,0.687046
8,2020,오프라인 판매,피부관리실/에스테틱,429462,0.000000
9,2021,오프라인 판매,피부관리실/에스테틱,573554,0.335517


In [ ]:
idx_zero = df_ch2[df_ch2['전년도 비율 차이'] == 0].index

df_ch2.loc[idx_zero, '전년도 비율 차이'] = None

# df_ch2

In [ ]:
df_ch2.to_csv('/content/drive/MyDrive/분석 연습/kaggle/연도별_화장품 채널별 매출.csv', index=False, header=True)

----

# 요일별 매출 독립성 검정

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/분석 연습/kaggle/perchase_per_day.csv")

df

,yr_m,day,day_week,count_perchase_per_w
0,2020-01,17,fri,1043
1,2020-01,18,sat,796
2,2020-01,19,sun,960
3,2020-01,20,mon,1147
4,2020-01,21,tue,1189
...,...,...,...,...
147,2019-10,27,sun,927
148,2019-10,28,mon,1019
149,2019-10,29,tue,944
150,2019-10,30,wed,912


In [ ]:
df_pivot = df.groupby(['yr_m', 'day_week'])['count_perchase_per_w'].sum()

In [ ]:
df_pivot = df_pivot.reset_index(level=1, drop=False)

In [ ]:
df_pivot = df_pivot.pivot_table(values='count_perchase_per_w', index='yr_m', columns='day_week')

In [ ]:
chi2_cont = chi2_contingency(observed=df_pivot, correction=False)

print(chi2_cont)

Chi2ContingencyResult(statistic=3691.0968752942936, pvalue=0.0, dof=24, expected_freq=array([[4418.63273111, 4345.84553999, 3554.75382618, 3706.51887159,
        4646.93688471, 4451.46200546, 4354.85014096],
       [5503.54279278, 5412.88411045, 4427.55508111, 4616.58310689,
        5787.90262885, 5544.43266239, 5424.09961754],
       [4342.03850046, 4270.5130297 , 3493.13439522, 3642.26868863,
        4566.38514455, 4374.29869989, 4279.36154155],
       [4868.15542729, 4787.96334502, 3916.3911519 , 4083.59577704,
        5119.6857472 , 4904.32453656, 4797.88401499],
       [4421.63054836, 4348.79397484, 3557.16554559, 3709.03355585,
        4650.0895947 , 4454.4820957 , 4357.80468496]]))


# 브랜드 분석

In [ ]:
# # runail 가격

# df_runail = pd.read_csv('/content/drive/MyDrive/분석 연습/kaggle/price_runail.csv')

# df_runail.info()
# df_runail.head()

## 브랜드 퍼널

In [ ]:
df_brand = pd.read_csv('/content/drive/MyDrive/분석 연습/kaggle/funnel_brand_new.csv')

df_brand.info()
df_brand.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4669 entries, 0 to 4668
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         4669 non-null   object
 1   brand        4649 non-null   object
 2   event_type   4669 non-null   object
 3   count_event  4669 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 146.0+ KB


,date,brand,event_type,count_event
0,2019-12,f.o.x,remove_from_cart,1116
1,2019-12,cnd,view,7666
2,2019-12,runail,cart,20055
3,2019-12,freedecor,view,4974
4,2019-12,NaN,view,172299


In [ ]:
# 특정 브랜드만 추출

List_extract = ['strong', 'marathon', 'runail', 'domix', 'ingarden']

df_b2 = df_brand[df_brand['brand'].isin(List_extract)]

df_b2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 2 to 4511
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         100 non-null    object
 1   brand        100 non-null    object
 2   event_type   100 non-null    object
 3   count_event  100 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 3.9+ KB


In [ ]:
# pivot

df_pv = df_b2.pivot(index=['date', 'brand'], columns='event_type', values='count_event')

df_pv.reset_index(level=[0,1], drop=False, inplace=True)

In [ ]:
# 컬럼 순서 변경

df_pv.columns.name = None

df_pv = df_pv[['date', 'brand', 'view', 'cart', 'remove_from_cart', 'purchase']]

df_pv.head()

,date,brand,view,cart,remove_from_cart,purchase
0,2019-10,domix,8279,5533,2709,2088
1,2019-10,ingarden,12729,8873,4995,2573
2,2019-10,marathon,2733,250,158,53
3,2019-10,runail,43223,22776,12310,7938
4,2019-10,strong,5133,627,321,144


In [ ]:
# 날짜별, 브랜드별로 정렬

df_pv.sort_values(by=['date', 'brand'], ascending=[True, True], inplace=True)

df_pv.head(10)

,date,brand,view,cart,remove_from_cart,purchase
0,2019-10,domix,8279,5533,2709,2088
1,2019-10,ingarden,12729,8873,4995,2573
2,2019-10,marathon,2733,250,158,53
3,2019-10,runail,43223,22776,12310,7938
4,2019-10,strong,5133,627,321,144
5,2019-11,domix,9169,5742,3094,2393
6,2019-11,ingarden,13964,9978,5996,3767
7,2019-11,marathon,3090,355,205,75
8,2019-11,runail,45612,24574,13822,10009
9,2019-11,strong,5307,801,406,198


## 브랜드 ARPPU

In [ ]:
# 월별, 브랜드별 arppu

brand_arppu = pd.read_csv('/content/drive/MyDrive/분석 연습/kaggle/arppu_brand.csv')

brand_arppu.info()
brand_arppu.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1260 entries, 0 to 1259
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    1260 non-null   object 
 1   brand   1255 non-null   object 
 2   ARPPU   1106 non-null   float64
 3   sales   1106 non-null   float64
dtypes: float64(2), object(2)
memory usage: 39.5+ KB


,date,brand,ARPPU,sales
0,2019-11,NaN,24.541823,619509.24
1,2019-11,freedecor,3.665456,7671.80
2,2019-11,grattol,13.835213,71472.71
3,2019-11,italwax,13.121360,24799.37
4,2019-11,severina,3.481502,6120.48


In [ ]:
# 특정 브랜드만 추출

brand_arppu = brand_arppu[brand_arppu['brand'].isin(List_extract)]

## join

In [ ]:
df_join = pd.merge(left=df_pv, right=brand_arppu, on=['date', 'brand'], how='inner', suffixes=['_x', '_y'])

df_join.info()
df_join.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              25 non-null     object 
 1   brand             25 non-null     object 
 2   view              25 non-null     int64  
 3   cart              25 non-null     int64  
 4   remove_from_cart  25 non-null     int64  
 5   purchase          25 non-null     int64  
 6   ARPPU             25 non-null     float64
 7   sales             25 non-null     float64
dtypes: float64(2), int64(4), object(2)
memory usage: 1.7+ KB


,date,brand,view,cart,remove_from_cart,purchase,ARPPU,sales
0,2019-10,domix,8279,5533,2709,2088,5.015350,10472.05
1,2019-10,ingarden,12729,8873,4995,2573,9.001706,23161.39
2,2019-10,marathon,2733,250,158,53,137.372642,7280.75
3,2019-10,runail,43223,22776,12310,7938,9.012255,71539.28
4,2019-10,strong,5133,627,321,144,202.754375,29196.63


In [ ]:
df_join.loc[0,'sales'] / df_join.loc[0,'purchase']

5.015349616858216

In [ ]:
# .xlsx로 저장
df_join.to_excel(dir_path+'brand_arppu.xlsx', sheet_name='ARPPU', header=True, index=False)

# ingarden remove에 영향을 미치는 브랜드?

## 전처리

In [ ]:
df_ig = pd.read_csv(dir_path+'cart_something_remove_ingarden.csv')

df_ig.info()
df_ig.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 836 entries, 0 to 835
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    836 non-null    object
 1   brand   836 non-null    object
 2   counts  836 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 19.7+ KB


,date,brand,counts
0,2019-10,runail,1234
1,2019-10,irisk,980
2,2019-10,masura,581
3,2019-10,grattol,522
4,2019-10,bpw.style,427


In [ ]:
date_origin = df_ig['date'].unique()

date_list = [re.sub(r'-', '_',i) for i in date_origin]

In [ ]:
# 월별로 변수 분리

for i in range(len(date_list)) :
  globals()['top5_counts_'+date_list[i]] = df_ig[df_ig['date']==date_origin[i]]
  globals()['top5_counts_'+date_list[i]].sort_values(by='counts', ascending=False, inplace=True)
  globals()['top5_counts_'+date_list[i]] = globals()['top5_counts_'+date_list[i]].head(5)

In [ ]:
# 다시 concat

df_ig2 = pd.concat([globals()['top5_counts_'+date_list[i]] for i in range(5)], axis=0)

df_ig2

,date,brand,counts
0,2019-10,runail,1234
1,2019-10,irisk,980
2,2019-10,masura,581
3,2019-10,grattol,522
4,2019-10,bpw.style,427
168,2019-11,runail,1520
169,2019-11,irisk,1129
170,2019-11,grattol,929
171,2019-11,masura,631
172,2019-11,bpw.style,583


In [ ]:
# 연도와 brand 순으로 정렬

df_ig2['date'] = pd.to_datetime(df_ig2['date'])

df_ig2.sort_values(by=['brand', 'date'], ascending=[False, True], inplace=True)

df_ig2.reset_index(drop=True, inplace=True)

df_ig2

,date,brand,counts
0,2020-01-01,uno,265
1,2020-02-01,uno,249
2,2019-10-01,runail,1234
3,2019-11-01,runail,1520
4,2019-12-01,runail,1059
5,2020-01-01,runail,1230
6,2020-02-01,runail,1143
7,2019-10-01,masura,581
8,2019-11-01,masura,631
9,2019-12-01,masura,382


In [ ]:
df_ig_g = df_ig2.groupby('brand').mean('counts').sort_values(by='counts', ascending=False)

df_ig_g['rate'] = round(df_ig_g['counts'] / 4987.2, 2)

df_ig_g

,counts,rate
brand,,
runail,1237.2,0.25
irisk,911.2,0.18
grattol,682.6,0.14
masura,523.4,0.10
bpw.style,446.0,0.09
uno,257.0,0.05


In [ ]:
df_ig_g['rate per remove'].sum()

0.81

## 연관규칙 분석 시도

In [ ]:
df_asc = pd.read_csv(dir_path+'raw_data_ingarden_association_rule.csv')

df_asc.info()
df_asc.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44035 entries, 0 to 44034
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     44035 non-null  object
 1   brand    44035 non-null  object
 2   session  44035 non-null  object
 3   counts   44035 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 1.3+ MB


,date,brand,session,counts
0,2019-10,runail,79ff3889-af61-4206-bf04-eb209aeb4054,1
1,2019-10,irisk,66978ef2-61a0-4f9c-b073-cea13e6ea65f,1
2,2019-10,solomeya,66978ef2-61a0-4f9c-b073-cea13e6ea65f,1
3,2019-10,s.care,79ff3889-af61-4206-bf04-eb209aeb4054,1
4,2019-10,swarovski,3110d1f7-fc28-4771-8b72-50b5ae8c7aa5,1


In [ ]:
# counts 확인

df_asc['counts'].unique()

array([1])

In [ ]:
# 새로운 데이터 프레임 생성

df_asc_new = df_asc.pivot_table(columns='brand', index='session', values='counts')

df_asc_new.reset_index(drop=True, inplace=True)

df_asc_new.replace({np.nan : 0}, inplace=True)

df_asc_new.head()

brand,airnails,almea,andrea,ardell,arganoil,art-visage,artex,aura,avene,balbcare,...,uskusi,veraclara,vichy,vilenta,vosev,yllozure,yoko,ypsed,zeitun,zinger
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# te = TransactionEncoder()
# te_ary = te.fit(df_asc_new).transform(df_asc_new)
# te_ary

In [ ]:
df_asc_new = df_asc_new.astype(bool)

df_asc_new.dtypes

brand
airnails    bool
almea       bool
andrea      bool
ardell      bool
arganoil    bool
            ... 
yllozure    bool
yoko        bool
ypsed       bool
zeitun      bool
zinger      bool
Length: 221, dtype: object

In [ ]:
# remove된 ingarden도 추가

df_asc_new["removed ingarden"] = True

df_asc_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15848 entries, 0 to 15847
Columns: 222 entries, airnails to removed ingarden
dtypes: bool(222)
memory usage: 3.4 MB


In [ ]:
# apriori

frequency_brand = apriori(df_asc_new, min_support=0.05, use_colnames=True)

frequency_brand

,support,itemsets
0,0.056790,(bluesky)
1,0.116355,(bpw.style)
2,0.056537,(de.lux)
3,0.072943,(domix)
4,0.068905,(freedecor)
5,0.223561,(grattol)
6,0.059692,(haruyama)
7,0.299218,(irisk)
8,0.053256,(lianail)
9,0.169233,(masura)


In [ ]:
asc_rules = association_rules(frequency_brand, metric="lift", min_threshold=1)

In [ ]:
asc_rules.info()
asc_rules.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   antecedents         94 non-null     object 
 1   consequents         94 non-null     object 
 2   antecedent support  94 non-null     float64
 3   consequent support  94 non-null     float64
 4   support             94 non-null     float64
 5   confidence          94 non-null     float64
 6   lift                94 non-null     float64
 7   leverage            94 non-null     float64
 8   conviction          94 non-null     float64
 9   zhangs_metric       94 non-null     float64
dtypes: float64(8), object(2)
memory usage: 7.5+ KB


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(removed ingarden),(bluesky),1.000000,0.056790,0.056790,0.056790,1.000000,0.000000,1.000000,0.000000
1,(bluesky),(removed ingarden),0.056790,1.000000,0.056790,1.000000,1.000000,0.000000,inf,0.000000
2,(bpw.style),(irisk),0.116355,0.299218,0.053004,0.455531,1.522409,0.018188,1.287094,0.388331
3,(irisk),(bpw.style),0.299218,0.116355,0.053004,0.177140,1.522409,0.018188,1.073871,0.489662
4,(runail),(bpw.style),0.410146,0.116355,0.062847,0.153231,1.316920,0.015124,1.043548,0.407987


In [ ]:
asc_rules[(asc_rules['consequents'].str.contains('removed ingarden', regex=False)) & (asc_rules['lift'] > 1)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
50,(bpw.style),"(removed ingarden, irisk)",0.116355,0.299218,0.053004,0.455531,1.522409,0.018188,1.287094,0.388331
51,(irisk),"(removed ingarden, bpw.style)",0.299218,0.116355,0.053004,0.177140,1.522409,0.018188,1.073871,0.489662
55,(runail),"(removed ingarden, bpw.style)",0.410146,0.116355,0.062847,0.153231,1.316920,0.015124,1.043548,0.407987
57,(bpw.style),"(runail, removed ingarden)",0.116355,0.410146,0.062847,0.540130,1.316920,0.015124,1.282653,0.272341
62,(grattol),"(removed ingarden, irisk)",0.223561,0.299218,0.068084,0.304544,1.017802,0.001191,1.007659,0.022526
63,(irisk),"(removed ingarden, grattol)",0.299218,0.223561,0.068084,0.227541,1.017802,0.001191,1.005152,0.024958
67,(masura),"(removed ingarden, grattol)",0.169233,0.223561,0.052814,0.312081,1.395950,0.014980,1.128677,0.341422
69,(grattol),"(masura, removed ingarden)",0.223561,0.169233,0.052814,0.236240,1.395950,0.014980,1.087734,0.365312
73,(runail),"(removed ingarden, grattol)",0.410146,0.223561,0.096731,0.235846,1.054951,0.005039,1.016076,0.088307
75,(grattol),"(runail, removed ingarden)",0.223561,0.410146,0.096731,0.432684,1.054951,0.005039,1.039727,0.067086


## 연관규칙 분석 시도2

In [40]:
df = pd.read_csv(dir_path+'cart_and_removed.csv')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 643092 entries, 0 to 643091
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   user_id        643092 non-null  int64 
 1   competitor     643092 non-null  object
 2   removed_brand  643092 non-null  object
 3   user_session   643092 non-null  object
 4   date           643092 non-null  object
dtypes: int64(1), object(4)
memory usage: 24.5+ MB


,user_id,competitor,removed_brand,user_session,date
0,407192905,ingarden,removed meisterwerk,88fa4a31-7291-4435-997b-43af1191d6a2,2019-12-14
1,446071693,staleks,removed glysolid,d97b0070-a11f-4076-bc85-28907379aa9d,2019-12-14
2,446071693,staleks,removed profhenna,d97b0070-a11f-4076-bc85-28907379aa9d,2019-12-14
3,446071693,staleks,removed luxvisage,d97b0070-a11f-4076-bc85-28907379aa9d,2019-12-14
4,446071693,staleks,removed bielenda,d97b0070-a11f-4076-bc85-28907379aa9d,2019-12-14


In [41]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

# df['date']

In [42]:
df_session = df['user_session'].unique()

df_user_id = df['user_id'].unique()

df_date = df['date'].unique()

print('session 수 : ',len(df_session))
print('user_id 수 : ',len(df_user_id))
print('date 수 : ',len(df_date))

session 수 :  96514
user_id 수 :  57921
date 수 :  92


In [43]:
df_dup = df[df.duplicated(subset=['user_id', 'user_session', 'date'], keep='last')][['user_id', 'user_session', 'date']]

df_dup.reset_index(drop=True, inplace=True)

df_dup.sort_values(by='date', ascending=True, inplace=True)

df_dup

,user_id,user_session,date
141807,405986628,12710291-238c-4492-9a07-7cb5c34ba956,2019-10-01
28024,401329661,9ad1b6fa-781b-45f1-970a-e4e9adf8ce38,2019-10-01
28025,512677005,5160e312-ea53-4346-ae6f-f9b3e80a2dac,2019-10-01
28026,549854453,a3b7a814-7b81-4484-b328-255fa537b3ee,2019-10-01
28027,523939534,e9146186-55b5-463c-869e-6a3bcd71fc1f,2019-10-01
...,...,...,...
502156,592638076,c8dbb96b-a29b-4dbe-b573-4a9daace20f4,2019-12-31
502155,595058034,8ded501d-a883-4006-ae24-618e09f145f1,2019-12-31
502154,592638076,c8dbb96b-a29b-4dbe-b573-4a9daace20f4,2019-12-31
100098,555410482,cb08cd6f-3600-4bb1-b24a-5769829fc63b,2019-12-31


In [2]:
# # 동일 세션별로, brand와 removed brand를 하나의 리스트로 저장

# brand_list = []

# cnt = 0

# for i in tqdm(range(df_dup.shape[0])) :
#   df_mini = df[(df['user_id'] == df_dup.iloc[i, 0]) & (df['user_session'] == df_dup.iloc[i, 1]) & (df['date'] == df_dup.iloc[i, 2])]
#   comp_set = set(df_mini['competitor'])
#   removed_set = set(df_mini['removed_brand'])

#   comp_set.update(removed_set)

#   comp_list = list(comp_set)

#   brand_list.append(comp_list)

# print(brand_list)

## 연관규칙 분석 시도3

In [7]:
df = pd.read_json(dir_path + 'cart_and_removed_json.json', orient='records', lines=True)

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100804 entries, 0 to 100803
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   user_id        100804 non-null  int64         
 1   user_session   100804 non-null  object        
 2   date           100804 non-null  datetime64[ns]
 3   competitor     100804 non-null  object        
 4   removed_brand  100804 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 3.8+ MB


,user_id,user_session,date,competitor,removed_brand
0,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,2019-10-01,[runail],[removed airnails]
1,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9,2019-10-01,"[lovely, lovely, lovely, lovely, lovely, lovel...","[removed runail, removed irisk, removed bpw.st..."
2,377667011,81326ac6-daa4-4f0a-b488-fd0956a78733,2019-10-01,[kapous],[removed estel]
3,524009100,8bbff347-0be1-470e-8860-d9a75db965b2,2019-10-01,[estel],[removed concept]
4,546705258,3b5c65c0-bb1c-453b-b340-4ebf973a3136,2019-10-01,"[grattol, grattol, grattol, irisk, de.lux]","[removed irisk, removed runail, removed de.lux..."


In [9]:
df['all'] = df['competitor'] + df['removed_brand']

df['all'] = df['all'].apply(lambda x : list(set(x)))

df['all']

0                                [runail, removed airnails]
1         [removed de.lux, removed nagaraku, nagaraku, r...
2                                   [kapous, removed estel]
3                                  [estel, removed concept]
4         [removed de.lux, grattol, removed irisk, irisk...
                                ...                        
100799                          [haruyama, removed grattol]
100800                         [removed kinetics, roubloff]
100801    [ingarden, removed domix, bluesky, grattol, ru...
100802                                  [removed oniq, cnd]
100803    [runail, removed irisk, removed ingarden, inga...
Name: all, Length: 100804, dtype: object

In [10]:
brand_all = df['all'].to_list()

brand_all

[['runail', 'removed airnails'],
 ['removed de.lux',
  'removed nagaraku',
  'nagaraku',
  'removed irisk',
  'lovely',
  'removed ingarden',
  'removed runail',
  'removed yoko',
  'removed jessnail',
  'removed milv',
  'removed lovely',
  'removed bpw.style'],
 ['kapous', 'removed estel'],
 ['estel', 'removed concept'],
 ['removed de.lux',
  'grattol',
  'removed irisk',
  'irisk',
  'removed runail',
  'de.lux'],
 ['removed depilflax', 'removed irisk', 'italwax', 'removed kapous'],
 ['nagaraku', 'pole', 'irisk', 'removed farmstay'],
 ['irisk', 'removed yoko'],
 ['smart',
  'runail',
  'rosi',
  'zinger',
  'removed runail',
  'haruyama',
  'removed smart',
  'removed zinger',
  'masura'],
 ['removed lianail',
  'runail',
  'removed ingarden',
  'removed estelare',
  'removed polarus',
  'removed uskusi',
  'removed grattol',
  'removed bpw.style',
  'removed aura',
  'removed haruyama',
  'removed runail',
  'removed yoko',
  'removed tertio',
  'removed zinger',
  'de.lux',
  'rem

In [11]:
te = TransactionEncoder()

te.fit(brand_all)

brand_te = te.transform(brand_all)

brand_te
# df_te = pd.DataFrame(brand_te)

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [14]:
brand_all_te = pd.DataFrame(data = brand_te, columns = te.columns_)

brand_all_te

,airnails,almea,andrea,ardell,art-visage,artex,aura,avene,babyliss,balbcare,...,vilenta,vosev,weaver,yllozure,yoko,ypsed,yu-r,zab,zeitun,zinger
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100799,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
100800,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
100801,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
100802,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [34]:
frequency_brand = apriori(brand_all_te, min_support=0.02, use_colnames=True)

frequency_brand

,support,itemsets
0,0.044998,(bluesky)
1,0.094877,(bpw.style)
2,0.033788,(concept)
3,0.030326,(cosmoprofi)
4,0.032320,(de.lux)
...,...,...
172,0.061595,"(runail, removed runail, removed irisk)"
173,0.032409,"(runail, removed runail, removed masura)"
174,0.020694,"(runail, removed runail, grattol, removed grat..."
175,0.050762,"(runail, removed runail, removed irisk, irisk)"


In [36]:
asc_rules = association_rules(frequency_brand, metric="lift", min_threshold=1)

asc_rules.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   antecedents         392 non-null    object 
 1   consequents         392 non-null    object 
 2   antecedent support  392 non-null    float64
 3   consequent support  392 non-null    float64
 4   support             392 non-null    float64
 5   confidence          392 non-null    float64
 6   lift                392 non-null    float64
 7   leverage            392 non-null    float64
 8   conviction          392 non-null    float64
 9   zhangs_metric       392 non-null    float64
dtypes: float64(8), object(2)
memory usage: 30.8+ KB


In [42]:
asc_rules[asc_rules['consequents'].str.contains('removed ingarden', regex=False)].sort_values(by=['lift', 'confidence'], ascending=[False, False])

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
209,(ingarden),"(irisk, removed ingarden)",0.113269,0.033649,0.020624,0.182081,5.411110,0.016813,1.181475,0.919327
235,(ingarden),"(runail, removed ingarden)",0.113269,0.044373,0.027142,0.239622,5.400139,0.022116,1.256778,0.918903
231,"(runail, ingarden)",(removed ingarden),0.048927,0.107327,0.027142,0.554745,5.168728,0.021891,2.004856,0.848020
227,(ingarden),"(removed runail, removed ingarden)",0.113269,0.040018,0.023382,0.206428,5.158357,0.018849,1.209698,0.909115
206,"(ingarden, irisk)",(removed ingarden),0.038044,0.107327,0.020624,0.542112,5.051028,0.016541,1.949545,0.833739
224,"(ingarden, removed runail)",(removed ingarden),0.044889,0.107327,0.023382,0.520884,4.853239,0.018564,1.863166,0.831267
62,(ingarden),(removed ingarden),0.113269,0.107327,0.051179,0.451830,4.209845,0.039022,1.628461,0.859857
329,(runail),"(removed runail, removed ingarden)",0.303480,0.040018,0.027013,0.089010,2.224240,0.014868,1.053779,0.790226
330,(removed runail),"(runail, removed ingarden)",0.274553,0.044373,0.027013,0.098388,2.217294,0.014830,1.059910,0.756774
233,(runail),"(ingarden, removed ingarden)",0.303480,0.051179,0.027142,0.089435,1.747513,0.011610,1.042014,0.614136
